# Multiple Linear Regression
God i hope this works

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold

from dataset_storing import cal_fire_temps

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


YEAR_                                                        2019
ALARM_DATE                                                    NaT
CAUSE                                                        16.0
GIS_ACRES                                                 2.29694
Shape_Length                                           935.160088
Shape_Area                                            9295.387634
geometry        MULTIPOLYGON (((213452.07079999894 -468768.277...
Name: 21138, dtype: object


In [2]:
cal_fire_temps.head(1)

,10_MONTHS_BEFORE,11_MONTHS_BEFORE,12_MONTHS_BEFORE,1_MONTHS_BEFORE,2_MONTHS_BEFORE,3_MONTHS_BEFORE,4_MONTHS_BEFORE,5_MONTHS_BEFORE,6_MONTHS_BEFORE,7_MONTHS_BEFORE,8_MONTHS_BEFORE,9_MONTHS_BEFORE,ALARM_DATE,CAUSE,GIS_ACRES,Shape_Area,Shape_Length,YEAR_,geometry
489,"[20.307, 20.325, 26.785, 20.325, 25.265, 20.30...","[21.683000000000003, 20.601, 26.483, 20.601, 2...","[19.936, 18.781, 21.254, 18.781, 20.896, 19.93...","[17.88, 15.861999999999998, 15.863, 15.8619999...","[17.609, 12.479, 11.754, 12.479, 14.687, 17.60...","[14.373, 9.202, 7.236000000000001, 9.202, 10.0...","[12.751, 8.165, 5.541, 8.165, 8.391, 12.751, 8...","[13.76, 10.718, 9.472, 10.718, 10.897, 13.76, ...","[15.414, 14.002, 14.239, 14.002, 14.809, 15.41...","[14.753, 14.117, 15.045, 14.117, 14.889, 14.75...","[16.518, 16.666, 20.099, 16.666, 18.107, 16.51...","[17.834, 18.829, 23.84800000000001, 18.829, 21...",2007-10-21,14.0,25.736713,104152.781054,1902.439051,2007,"MULTIPOLYGON (((138036.906 -402646.363, 138086..."


Separating inputs (`fire_X`) and outputs (`fire_x`)

In [3]:
fire_X = cal_fire_temps.loc[cal_fire_temps.index, ["ALARM_DATE", "CAUSE", "YEAR_", "1_MONTHS_BEFORE", "2_MONTHS_BEFORE", "3_MONTHS_BEFORE", "4_MONTHS_BEFORE", "5_MONTHS_BEFORE", "6_MONTHS_BEFORE", "7_MONTHS_BEFORE", "8_MONTHS_BEFORE", "9_MONTHS_BEFORE", "10_MONTHS_BEFORE", "11_MONTHS_BEFORE", "12_MONTHS_BEFORE"]]
fire_y = cal_fire_temps.loc[cal_fire_temps.index, ["geometry"]]

In [4]:
fire_train_1, fire_test = train_test_split(fire_X, train_size=.8, random_state=1)
fire_train_1

,ALARM_DATE,CAUSE,YEAR_,1_MONTHS_BEFORE,2_MONTHS_BEFORE,3_MONTHS_BEFORE,4_MONTHS_BEFORE,5_MONTHS_BEFORE,6_MONTHS_BEFORE,7_MONTHS_BEFORE,8_MONTHS_BEFORE,9_MONTHS_BEFORE,10_MONTHS_BEFORE,11_MONTHS_BEFORE,12_MONTHS_BEFORE
14280,1995-09-10,4.0,1995,"[20.683000000000003, 19.103, 22.402, 19.103, 2...","[18.148, 16.072, 16.084999999999994, 16.072, 1...","[13.79, 9.225, 7.613999999999999, 9.225, 9.449...","[13.492, 7.291, 6.227, 7.291, 9.187, 13.492, 7...","[13.524, 10.459, 8.899000000000003, 10.459, 9....","[16.212, 11.938, 11.11, 11.938, 13.15400000000...","[14.63, 11.92, 11.582, 11.92, 12.103, 14.63, 1...","[14.42, 13.4, 13.567, 13.4, 13.519, 14.42, 13....","[14.342, 14.974, 16.766, 14.974, 15.169, 14.34...","[16.400000000000002, 17.797, 21.30800000000000...","[19.114, 20.534, 25.541, 20.534, 24.242, 19.11...","[21.05, 20.442, 26.026, 20.442, 25.022, 21.05,..."
17699,2012-08-13,1.0,2012,"[20.003, 19.389, 25.898000000000003, 19.389, 2...","[19.904, 20.54, 24.503, 20.54, 22.728, 19.904,...","[18.692, 17.294999999999998, 17.80700000000000...","[15.455, 11.478, 10.014, 11.478, 11.643, 15.45...","[13.258, 9.118, 6.391, 9.118, 8.86799999999999...","[15.113, 9.993, 8.607999999999999, 9.993, 11.5...","[13.664, 11.116, 9.392, 11.116, 10.60900000000...","[13.669, 11.503, 11.264, 11.503, 11.576, 13.66...","[15.094, 14.15, 14.645, 14.15, 14.751, 15.094,...","[16.644000000000002, 16.555, 19.772, 16.555, 1...","[17.34, 18.482, 23.024, 18.482, 21.066, 17.34,...","[18.984, 19.632, 26.095, 19.632, 23.851, 18.98..."
3867,1912-02-01,14.0,1912,"[11.225, 8.124, 6.598999999999998, 8.124, 7.94...","[14.268, 12.435, 12.151, 12.435, 12.3459999999...","[13.305, 12.502, 12.984000000000002, 12.502, 1...","[14.432, 14.389, 15.823999999999998, 14.389, 1...","[15.842, 16.811999999999998, 21.385, 16.811999...","[18.419, 19.296, 25.968000000000004, 19.296, 2...","[19.073, 17.675, 23.056, 17.675, 22.502, 19.07...","[18.57, 16.655, 19.265, 16.655, 19.214, 18.57,...","[17.227, 15.159, 15.535999999999998, 15.159, 1...","[16.42, 11.576, 9.924, 11.576, 12.505999999999...","[12.322, 8.122, 5.433, 8.122, 7.728, 12.322, 8...","[14.305, 9.238, 7.586, 9.238, 9.767, 14.305, 9..."
18814,1979-07-17,1.0,1979,"[19.088, 19.834, 25.951, 19.834, 23.811, 19.08...","[19.371, 20.242, 25.447, 20.242, 23.416, 19.37...","[20.53, 19.705, 20.709, 19.705, 21.332, 20.53,...","[19.483, 17.171999999999993, 19.246, 17.171999...","[14.752, 10.865, 9.547, 10.865, 11.239, 14.752...","[12.201, 6.692, 4.623, 6.692, 7.37599999999999...","[12.188, 8.027999999999999, 6.267, 8.027999999...","[11.978, 9.665, 7.938, 9.665, 8.541, 11.978, 9...","[13.004, 12.347, 11.427, 12.347, 11.438, 13.00...","[14.26, 13.407, 13.739, 13.407, 13.908, 14.26,...","[15.909, 17.229, 19.566, 17.229, 17.5540000000...","[18.529, 18.983, 23.489, 18.983, 21.734, 18.52..."
13335,1984-06-24,7.0,1984,"[16.782, 18.536, 22.206000000000003, 18.536, 1...","[19.09, 19.826, 23.94, 19.826, 23.01, 19.09, 1...","[21.687, 20.89, 25.723000000000003, 20.89, 24....","[21.868, 20.955, 23.734, 20.955, 23.513, 21.86...","[19.842, 17.625999999999994, 17.642, 17.625999...","[16.073, 11.972, 10.936, 11.972, 12.307, 16.07...","[14.232, 10.294, 8.597000000000001, 10.294, 10...","[14.934, 9.192, 7.95, 9.192, 10.851, 14.934, 9...","[13.962, 10.353000000000002, 9.276, 10.3530000...","[15.021, 13.705, 13.045, 13.705, 13.825, 15.02...","[14.843, 13.105, 13.318, 13.105, 14.114, 14.84...","[17.629, 17.679000000000002, 20.945, 17.679000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16534,2008-06-23,1.0,2008,"[17.834, 18.829, 23.84800000000001, 18.829, 21...","[20.307, 20.325, 26.785, 20.325, 25.265, 20.30...","[21.683000000000003, 20.601, 26.483, 20.601, 2...","[19.936, 18.781, 21.254, 18.781, 20.896, 19.93...","[18.762, 15.439999999999998, 16.168, 15.439999...","[16.98, 13.278, 13.017, 13.278, 14.4, 16.98, 1...","[12.935, 8.318999999999999, 5.96, 8.3189999999...","[12.838, 8.026, 6.731, 8.026, 8.196, 12.838, 8...","[13.18, 10.4490000000000

In [5]:
fire_train_2, fire_test2 = train_test_split(fire_X, train_size=.8, random_state=1)
fire_train_2

,ALARM_DATE,CAUSE,YEAR_,1_MONTHS_BEFORE,2_MONTHS_BEFORE,3_MONTHS_BEFORE,4_MONTHS_BEFORE,5_MONTHS_BEFORE,6_MONTHS_BEFORE,7_MONTHS_BEFORE,8_MONTHS_BEFORE,9_MONTHS_BEFORE,10_MONTHS_BEFORE,11_MONTHS_BEFORE,12_MONTHS_BEFORE
14280,1995-09-10,4.0,1995,"[20.683000000000003, 19.103, 22.402, 19.103, 2...","[18.148, 16.072, 16.084999999999994, 16.072, 1...","[13.79, 9.225, 7.613999999999999, 9.225, 9.449...","[13.492, 7.291, 6.227, 7.291, 9.187, 13.492, 7...","[13.524, 10.459, 8.899000000000003, 10.459, 9....","[16.212, 11.938, 11.11, 11.938, 13.15400000000...","[14.63, 11.92, 11.582, 11.92, 12.103, 14.63, 1...","[14.42, 13.4, 13.567, 13.4, 13.519, 14.42, 13....","[14.342, 14.974, 16.766, 14.974, 15.169, 14.34...","[16.400000000000002, 17.797, 21.30800000000000...","[19.114, 20.534, 25.541, 20.534, 24.242, 19.11...","[21.05, 20.442, 26.026, 20.442, 25.022, 21.05,..."
17699,2012-08-13,1.0,2012,"[20.003, 19.389, 25.898000000000003, 19.389, 2...","[19.904, 20.54, 24.503, 20.54, 22.728, 19.904,...","[18.692, 17.294999999999998, 17.80700000000000...","[15.455, 11.478, 10.014, 11.478, 11.643, 15.45...","[13.258, 9.118, 6.391, 9.118, 8.86799999999999...","[15.113, 9.993, 8.607999999999999, 9.993, 11.5...","[13.664, 11.116, 9.392, 11.116, 10.60900000000...","[13.669, 11.503, 11.264, 11.503, 11.576, 13.66...","[15.094, 14.15, 14.645, 14.15, 14.751, 15.094,...","[16.644000000000002, 16.555, 19.772, 16.555, 1...","[17.34, 18.482, 23.024, 18.482, 21.066, 17.34,...","[18.984, 19.632, 26.095, 19.632, 23.851, 18.98..."
3867,1912-02-01,14.0,1912,"[11.225, 8.124, 6.598999999999998, 8.124, 7.94...","[14.268, 12.435, 12.151, 12.435, 12.3459999999...","[13.305, 12.502, 12.984000000000002, 12.502, 1...","[14.432, 14.389, 15.823999999999998, 14.389, 1...","[15.842, 16.811999999999998, 21.385, 16.811999...","[18.419, 19.296, 25.968000000000004, 19.296, 2...","[19.073, 17.675, 23.056, 17.675, 22.502, 19.07...","[18.57, 16.655, 19.265, 16.655, 19.214, 18.57,...","[17.227, 15.159, 15.535999999999998, 15.159, 1...","[16.42, 11.576, 9.924, 11.576, 12.505999999999...","[12.322, 8.122, 5.433, 8.122, 7.728, 12.322, 8...","[14.305, 9.238, 7.586, 9.238, 9.767, 14.305, 9..."
18814,1979-07-17,1.0,1979,"[19.088, 19.834, 25.951, 19.834, 23.811, 19.08...","[19.371, 20.242, 25.447, 20.242, 23.416, 19.37...","[20.53, 19.705, 20.709, 19.705, 21.332, 20.53,...","[19.483, 17.171999999999993, 19.246, 17.171999...","[14.752, 10.865, 9.547, 10.865, 11.239, 14.752...","[12.201, 6.692, 4.623, 6.692, 7.37599999999999...","[12.188, 8.027999999999999, 6.267, 8.027999999...","[11.978, 9.665, 7.938, 9.665, 8.541, 11.978, 9...","[13.004, 12.347, 11.427, 12.347, 11.438, 13.00...","[14.26, 13.407, 13.739, 13.407, 13.908, 14.26,...","[15.909, 17.229, 19.566, 17.229, 17.5540000000...","[18.529, 18.983, 23.489, 18.983, 21.734, 18.52..."
13335,1984-06-24,7.0,1984,"[16.782, 18.536, 22.206000000000003, 18.536, 1...","[19.09, 19.826, 23.94, 19.826, 23.01, 19.09, 1...","[21.687, 20.89, 25.723000000000003, 20.89, 24....","[21.868, 20.955, 23.734, 20.955, 23.513, 21.86...","[19.842, 17.625999999999994, 17.642, 17.625999...","[16.073, 11.972, 10.936, 11.972, 12.307, 16.07...","[14.232, 10.294, 8.597000000000001, 10.294, 10...","[14.934, 9.192, 7.95, 9.192, 10.851, 14.934, 9...","[13.962, 10.353000000000002, 9.276, 10.3530000...","[15.021, 13.705, 13.045, 13.705, 13.825, 15.02...","[14.843, 13.105, 13.318, 13.105, 14.114, 14.84...","[17.629, 17.679000000000002, 20.945, 17.679000..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16534,2008-06-23,1.0,2008,"[17.834, 18.829, 23.84800000000001, 18.829, 21...","[20.307, 20.325, 26.785, 20.325, 25.265, 20.30...","[21.683000000000003, 20.601, 26.483, 20.601, 2...","[19.936, 18.781, 21.254, 18.781, 20.896, 19.93...","[18.762, 15.439999999999998, 16.168, 15.439999...","[16.98, 13.278, 13.017, 13.278, 14.4, 16.98, 1...","[12.935, 8.318999999999999, 5.96, 8.3189999999...","[12.838, 8.026, 6.731, 8.026, 8.196, 12.838, 8...","[13.18, 10.4490000000000